# Fully Convolutional Interior/Edge Segmentation for 2D Data

---

Classifies each pixel as either Cell Edge, Cell Interior, or Background.

There are 2 different Cell Edge classes (Cell-Cell Boundary and Cell-Background Boundary)

In [1]:
import os
import errno

import numpy as np

import deepcell

In [2]:
from deepcell.utils.data_utils import get_data
from tensorflow.keras.utils import get_file

def load_data(path, mode, test_size=0.2, seed=0):
    """Loads dataset.
    Args:
         test_size (float): fraction of data to reserve as test data
        seed (int): the seed for randomly shuffling the dataset
    Returns:
           tuple: (x_train, y_train), (x_test, y_test).
    """
    #basepath = os.path.expanduser(os.path.join('~', '.keras', 'datasets'))
    #prefix = path.split(os.path.sep)[:-1]
    #data_dir = os.path.join(basepath, *prefix) if prefix else basepath
    #if not os.path.exists(data_dir):
    #    os.makedirs(data_dir)
    #elif not os.path.isdir(data_dir):
    #    raise IOError('{} exists but is not a directory'.format(data_dir))

    train_dict, test_dict = get_data(
        path,
        mode=mode,
        test_size=test_size,
        seed=seed)
    x_train, y_train = train_dict['X'], train_dict['y']
    x_test, y_test = test_dict['X'], test_dict['y']
    return (x_train, y_train), (x_test, y_test)

## Load the data

### Download the data from `deepcell.datasets`

`deepcell.datasets` provides access to a set of annotated live-cell imaging datasets which can be used for training cell segmentation and tracking models.
All dataset objects share the `load_data()` method, which allows the user to specify the name of the file (`path`), the fraction of data reserved for testing (`test_size`) and a `seed` which is used to generate the random train-test split.
Metadata associated with the dataset can be accessed through the `metadata` attribute.

In [3]:
# Download the data (saves to ~/.keras/datasets)
#filename = 'HeLa_S3.npz'
#test_size = 0.2 # % of data saved as test
#seed = 0 # seed for random train-test split

#(X_train, y_train), (X_test, y_test) = deepcell.datasets.hela_s3.load_data(filename, test_size=test_size, seed=seed)

#print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

In [4]:

#file = np.load('../train.npz')
#file['X'].shape
#X = np.vstack(file['X'])
#X = X[0:-1:250]
#print(X.shape)

#y = np.vstack(file['y'])[0:-1:250]
#print(y.shape)

#np.savez("../seg_train.npz", X=X, y=y)

In [3]:
#data = np.load('../seg_train.npz')

(X_train, y_train), (X_test, y_test) = load_data('../seg_train.npz', mode='sample')
print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

X.shape: (140, 530, 530, 1)
y.shape: (140, 530, 530, 1)


In [4]:
X_train.dtype

dtype('int32')

### Set up filepath constants

In [21]:
# Set up other required filepaths

PREFIX = 'test'

ROOT_DIR = '../seg_training/attempt_2'  
MODEL_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'models', PREFIX))
LOG_DIR = os.path.abspath(os.path.join(ROOT_DIR, 'logs', PREFIX))

# create directories if they do not exist
for d in (MODEL_DIR, LOG_DIR):
    try:
        os.makedirs(d)
    except OSError as exc:  # Guard against race condition
        if exc.errno != errno.EEXIST:
            raise

## Create the Foreground/Background FeatureNet Model

Here we instantiate two `FeatureNet` models from `deepcell.model_zoo` for foreground/background separation as well as the interior/edge segmentation.

In [6]:
norm_method = 'std'  # data normalization
receptive_field = 61  # should be adjusted for the scale of the data  ## 61
n_skips = 1  # number of skip-connections (only for FC training)

In [7]:
# pixelwise transform settings
dilation_radius = 1  # change dilation radius for edge dilation
separate_edge_classes = True  # break edges into cell-background edge, cell-cell edge
pixelwise_kwargs = {
    'dilation_radius': dilation_radius,
    'separate_edge_classes': separate_edge_classes,
}

In [8]:
X_train.shape[1:]

(530, 530, 1)

In [9]:
X_train = X_train.astype(np.float32)
X_test = X_test.astype(np.float32)

In [10]:
from deepcell import model_zoo

#fgbg_model = model_zoo.bn_feature_net_skip_2D(
#    n_features=2,  # segmentation mask (is_cell, is_not_cell)
#    receptive_field=receptive_field,
#    norm_method=norm_method,
#    n_skips=n_skips,
#    n_conv_filters=32,
#    n_dense_filters=128,
#    input_shape=tuple(X_train.shape[1:]),
#    last_only=False)

from deepcell import model_zoo

input_shape=(530, 530, 1)

fgbg_model = model_zoo.bn_feature_net_skip_2D(
    n_features=2,  # segmentation mask (is_cell, is_not_cell)
    receptive_field=receptive_field,
    norm_method=norm_method,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    input_shape=input_shape,
    last_only=False)

2021-08-24 01:52:30.384868: W tensorflow/python/util/util.cc:348] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.
2021-08-24 01:52:30.389676: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN)to use the following CPU instructions in performance-critical operations:  AVX512F
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-08-24 01:52:30.414556: I tensorflow/core/platform/profile_utils/cpu_utils.cc:104] CPU Frequency: 1999995000 Hz
2021-08-24 01:52:30.418777: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x55974cd8a150 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2021-08-24 01:52:30.418802: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2021-08-24 01:52:30.420999: I tensorflow/co

## Prepare for training

### Set up training parameters.

There are a number of tunable hyper parameters necessary for training deep learning models:

**model_name**: Incorporated into any files generated during the training process.

**n_epoch**: The number of complete passes through the training dataset.

**lr**: The learning rate determines the speed at which the model learns. Specifically it controls the relative size of the updates to model values after each batch.

**optimizer**: The TensorFlow module [tf.keras.optimizers](https://www.tensorflow.org/api_docs/python/tf/keras/optimizers) offers optimizers with a variety of algorithm implementations. DeepCell typically uses the Adam or the SGD optimizers.

**lr_sched**: A learning rate scheduler allows the learning rate to adapt over the course of model training. Typically a larger learning rate is preferred during the start of the training process, while a small learning rate allows for fine-tuning during the end of training.

**batch_size**: The batch size determines the number of samples that are processed before the model is updated. The value must be greater than one and less than or equal to the number of samples in the training dataset.

In [11]:
from tensorflow.keras.optimizers import SGD
from deepcell.utils.train_utils import rate_scheduler

fgbg_model_name = 'conv_fgbg_model'
pixelwise_model_name = 'conv_edgeseg_model'

n_epoch = 5  # Number of training epochs

lr = 0.01
fgbg_optimizer = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)
pixelwise_optimizer = SGD(lr=lr, decay=1e-6, momentum=0.9, nesterov=True)

lr_sched = rate_scheduler(lr=lr, decay=0.99)

batch_size = 1  # fully convolutional training uses 1 image per batch

### Create the DataGenerators

The `ImageFullyConvDataGenerator` outputs a raw image (`X`) with it's labeled annotation mask (`y`). Additionally, it can apply a transform to `y` to change the task the model learns. Below we generate 2 training and validation data sets for both the foreground/background model and the pixelwise model.

In [12]:
from deepcell.image_generators import ImageFullyConvDataGenerator

test_size = 0.2
seed = 0

#datagen = ImageFullyConvDataGenerator(
#    rotation_range=180,
#    zoom_range=(.8, 1.2),
#    horizontal_flip=True,
#    vertical_flip=True)

datagen = ImageFullyConvDataGenerator(
    rotation_range=30,
    zoom_range=(.9, 1.1),
    horizontal_flip=False,
    vertical_flip=False)

datagen_val = ImageFullyConvDataGenerator()

In [22]:
# Create the foreground/background data iterators

fgbg_train_data = datagen.flow(
    {'X': X_train, 'y': y_train},
    seed=seed,
    skip=n_skips,
    transform='fgbg',
    batch_size=batch_size)

fgbg_val_data = datagen_val.flow(
    {'X': X_test, 'y': y_test},
    seed=seed,
    skip=n_skips,
    transform='fgbg',
    batch_size=batch_size)

# Skipping this step if fgbg trained

In [13]:
# Create the pixelwise data iterators

pixelwise_train_data = datagen.flow(
    {'X': X_train, 'y': y_train},
    seed=seed,
    skip=n_skips,
    transform='pixelwise',
    transform_kwargs=pixelwise_kwargs,
    batch_size=batch_size)

pixelwise_val_data = datagen_val.flow(
    {'X': X_test, 'y': y_test},
    seed=seed,
    skip=n_skips,
    transform='pixelwise',
    transform_kwargs=pixelwise_kwargs,
    batch_size=batch_size)

Visualize the data generator output.

In [14]:
from matplotlib import pyplot as plt

# Different data generators but same data and same seed
img, fgbg_output = fgbg_train_data.next()
_, pixelwise_output = pixelwise_train_data.next()

if n_skips:
    fgbg_output = fgbg_output[0]
    pixelwise_output = pixelwise_output[0]

fig, axes = plt.subplots(1, 3, figsize=(15, 15))
ax = axes.ravel()

ax[0].imshow(img[0, ..., 0])
ax[0].set_title('Source Image')

ax[1].imshow(np.argmax(fgbg_output[0, ...], axis=-1))
ax[1].set_title('Foreground/Background')

ax[2].imshow(np.argmax(pixelwise_output[0, ...], axis=-1))
ax[2].set_title('Pixelwise')

plt.show()

NameError: name 'fgbg_train_data' is not defined

### Compile the model with a loss function

Each model is trained with it's own loss function. `weighted_categorical_crossentropy` is often used for classification models, but `weighted_focal_loss` is also supported. The losses are passed to `model.compile` before training.

In [25]:
from deepcell import losses

def loss_function(y_true, y_pred):
    return losses.weighted_categorical_crossentropy(
        y_true, y_pred,
        n_classes=2,
        from_logits=False)

fgbg_model.compile(
    loss=loss_function,
    optimizer=fgbg_optimizer,
    metrics=['accuracy'])

## Train the foreground/background model

Call `fit()` on the compiled model, along with a default set of callbacks.

In [26]:
from deepcell.utils.train_utils import get_callbacks
from deepcell.utils.train_utils import count_gpus

model_path = os.path.join(MODEL_DIR, '{}.h5'.format(fgbg_model_name))
loss_path = os.path.join(MODEL_DIR, '{}.npz'.format(fgbg_model_name))

num_gpus = count_gpus()

print('Training on', num_gpus, 'GPUs.')

train_callbacks = get_callbacks(
    model_path,
    lr_sched=lr_sched,
    save_weights_only=num_gpus >= 2,
    monitor='val_loss',
    verbose=1)

loss_history = fgbg_model.fit(  # Seems like fgbg is overfitting - removed all features! 
    fgbg_train_data,
    steps_per_epoch=fgbg_train_data.y.shape[0] // batch_size,
    epochs=3,
    validation_data=fgbg_val_data,
    validation_steps=fgbg_val_data.y.shape[0] // batch_size,
    callbacks=train_callbacks)

Training on 0 GPUs.
Epoch 1/3
140/140 [==============================] - ETA: 0s - loss: 0.6133 - functional_5_loss: 0.3008 - functional_7_loss: 0.2945 - functional_5_accuracy: 0.8260 - functional_7_accuracy: 0.8308 
Epoch 00001: val_loss improved from inf to 1.19154, saving model to /home/jupyter/seg_training/attempt_1/models/test/conv_fgbg_model.h5
140/140 [==============================] - 1855s 13s/step - loss: 0.6133 - functional_5_loss: 0.3008 - functional_7_loss: 0.2945 - functional_5_accuracy: 0.8260 - functional_7_accuracy: 0.8308 - val_loss: 1.1915 - val_functional_5_loss: 0.5381 - val_functional_7_loss: 0.6353 - val_functional_5_accuracy: 0.6369 - val_functional_7_accuracy: 0.5986
Epoch 2/3
140/140 [==============================] - ETA: 0s - loss: 0.5430 - functional_5_loss: 0.2666 - functional_7_loss: 0.2583 - functional_5_accuracy: 0.8365 - functional_7_accuracy: 0.8467 
Epoch 00002: val_loss improved from 1.19154 to 0.76091, saving model to /home/jupyter/seg_training/att

In [27]:
fgbg_model.save("../fgbg_model.h5")

## Create the `pixelwise` FeatureNet Model

Here we instantiate two `FeatureNet` models from `deepcell.model_zoo` for foreground/background separation as well as the interior/edge segmentation.

In [15]:
# load the fgbg_model
fgbg_model.load_weights("../seg_training/attempt_1/models/test/conv_fgbg_model.h5")

In [16]:
X_train.shape

(140, 530, 530, 1)

In [17]:
pixelwise_model = model_zoo.bn_feature_net_skip_2D(
    fgbg_model=fgbg_model,
    n_features=4 if separate_edge_classes else 3,
    receptive_field=receptive_field,
    norm_method=norm_method,
    n_skips=n_skips,
    n_conv_filters=32,
    n_dense_filters=128,
    last_only=False,
    input_shape=tuple(X_train.shape[1:]))

### Compile the model with a loss function

Just like the foreground/background model, the `pixelwise` model is compiled with the `weighted_categorical_crossentropy` loss function.

In [18]:
from deepcell import losses

def loss_function(y_true, y_pred):
    return losses.weighted_categorical_crossentropy(
        y_true, y_pred,
        n_classes=4 if separate_edge_classes else 3,
        from_logits=False)

pixelwise_model.compile(
    loss=loss_function,
    optimizer=pixelwise_optimizer,
    metrics=['accuracy'])

## Train the `pixelwise` model

Call `fit()` on the compiled model, along with a default set of callbacks.

In [22]:
from deepcell.utils.train_utils import get_callbacks
from deepcell.utils.train_utils import count_gpus

model_path = os.path.join(MODEL_DIR, '{}.h5'.format(pixelwise_model_name))
loss_path = os.path.join(MODEL_DIR, '{}.npz'.format(pixelwise_model_name))

num_gpus = count_gpus()

print('Training on', num_gpus, 'GPUs.')

train_callbacks = get_callbacks(
    model_path,
    lr_sched=lr_sched,
    save_weights_only=num_gpus >= 2,
    monitor='val_loss',
    verbose=1)

loss_history = pixelwise_model.fit(
    pixelwise_train_data,
    steps_per_epoch=pixelwise_train_data.y.shape[0] // batch_size,
    epochs=5,
    validation_data=pixelwise_val_data,
    validation_steps=pixelwise_val_data.y.shape[0] // batch_size,
    callbacks=train_callbacks)

Training on 0 GPUs.
Epoch 1/5
140/140 [==============================] - ETA: 0s - loss: 0.9915 - functional_7_loss: 0.4765 - functional_9_loss: 0.4787 - functional_7_accuracy: 0.7808 - functional_9_accuracy: 0.7856 
Epoch 00001: val_loss improved from inf to 1.13870, saving model to /home/jupyter/seg_training/attempt_2/models/test/conv_edgeseg_model.h5
140/140 [==============================] - 2926s 21s/step - loss: 0.9915 - functional_7_loss: 0.4765 - functional_9_loss: 0.4787 - functional_7_accuracy: 0.7808 - functional_9_accuracy: 0.7856 - val_loss: 1.1387 - val_functional_7_loss: 0.5540 - val_functional_9_loss: 0.5485 - val_functional_7_accuracy: 0.6294 - val_functional_9_accuracy: 0.6293
Epoch 2/5
140/140 [==============================] - ETA: 0s - loss: 0.8073 - functional_7_loss: 0.3867 - functional_9_loss: 0.3844 - functional_7_accuracy: 0.8414 - functional_9_accuracy: 0.8406 
Epoch 00002: val_loss improved from 1.13870 to 1.07816, saving model to /home/jupyter/seg_training/

In [ ]:
pixelwise_model.save("../pixelwise_model.h5")

## Predict on test data

Use the trained model to predict on new data and post-process the results into a label mask.

In [ ]:
pixelwise_model

In [ ]:
test_images = pixelwise_model.predict(X_test)[-1]
test_images_fgbg = fgbg_model.predict(X_test)[-1]

print('watershed transform shape:', test_images.shape)
print('segmentation mask shape:', test_images_fgbg.shape)

#### Post-processing

In [ ]:
# threshold the foreground/background
# and remove back ground from edge transform
threshold = 0.8

fg_thresh = test_images_fgbg[..., 1] > threshold
fg_thresh = np.expand_dims(fg_thresh, axis=-1)

test_images_post_fgbg = test_images * fg_thresh

In [ ]:
# Label interior predictions
from skimage.measure import label
from skimage import morphology

labeled_images = []
for i in range(test_images_post_fgbg.shape[0]):
    interior = test_images_post_fgbg[i, ..., 2] > .5
    labeled_image = label(interior)
    labeled_image = morphology.remove_small_objects(
        labeled_image, min_size=130, connectivity=1)  ## min_size=50
    labeled_images.append(labeled_image)
labeled_images = np.array(labeled_images)
labeled_images = np.expand_dims(labeled_images, axis=-1)  

print('labeled_images shape:', labeled_images.shape)

In [ ]:
test_images.shape

In [ ]:
import matplotlib.pyplot as plt

#index = np.random.randint(low=0, high=X_test.shape[0])

index = 3
print('Image number:', index)

fig, axes = plt.subplots(ncols=3, nrows=2, figsize=(15, 15), sharex=True, sharey=True)
ax = axes.ravel()

ax[0].imshow(X_test[index, ..., 0])
ax[0].set_title('Source Image')

ax[1].imshow(test_images_fgbg[index, ..., 1])
ax[1].set_title('Segmentation Prediction')

ax[2].imshow(fg_thresh[index, ..., 0], cmap='jet')
ax[2].set_title('FGBG Threshold {}%'.format(threshold * 100))

ax[3].imshow(test_images[index, ..., 0] + test_images[index, ..., 1], cmap='jet')
ax[3].set_title('Edge Prediction')

ax[4].imshow(test_images[index, ..., 2], cmap='jet')
ax[4].set_title('Interior Prediction')

ax[5].imshow(labeled_images[index, ..., 0], cmap='jet')
ax[5].set_title('Instance Segmentation')



fig.tight_layout()
plt.show()

In [ ]:
plt.imshow(labeled_images[3])

In [ ]:
TEST_FILE = "../validation.trks"

(X_train, y_train), (X_test, y_test) = load_data('../validation.npz', mode='sample', test_size=1)
print('X.shape: {}\ny.shape: {}'.format(X_train.shape, y_train.shape))

In [ ]:
print('y.shape: {}\ny.shape: {}'.format(X_test.shape, y_test.shape))

In [ ]:
# Need to stack them together
## Store them into seg_validation.npz

val_all = np.load('../validation.npz')
X = val_all['X']
y = val_all['y']